In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display 
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)


font= {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }
%config Completer.use_jedi = False

In [ ]:
df = pd.read_csv("C:\Users\MEGHA\Downloads\Project-Housing--2---1---1- (1)\Project-Housing_splitted\train.csv")

display(df.head())


In [ ]:
display(df.shape)


In [ ]:
display(df.info())


In [ ]:
display(df.describe(include="all"))

In [ ]:
# Checking for null percentages
display(round(df.isnull().sum()/len(df.index) * 100,2))

In [ ]:
# removing features having high NA's (> 90%) and ID
df = df.drop(["PoolQC","Fence","MiscFeature","Alley","Id"],axis=1,errors="ignore")

In [ ]:
#Checking values in Garage
df["GarageType"].value_counts(dropna=False)

In [ ]:
#Checking features where garage values are null
garageVariables =  ["GarageType", "GarageYrBlt", "GarageFinish", "GarageQual","GarageCond"]
garage = df[df["GarageType"].isnull()]
display(garage.shape)
garage.describe(include="all")

In [ ]:
#Checking features where garage values are not null
garage_not_null = df[~df["GarageType"].isnull()]
des = garage_not_null[['Street', 'Utilities','LandSlope', 'Condition2', 'RoofMatl','RoofStyle', 'MasVnrType', 'ExterQual', 'Heating', 'Functional'] + garageVariables].describe(include="object")
display(f'shape of the frame {garage_not_null.shape}')
display(garage_not_null['GarageType'].value_counts())
display(garage_not_null['GarageFinish'].value_counts())
display(des)

In [ ]:
#Checking values in basement
df["BsmtQual"].value_counts(dropna=False)

In [ ]:
#Checking features where basement values are null
basementVariables =  ['BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtFinType1', 'BsmtCond']
basement = df[df["BsmtQual"].isnull()]
display(basement.shape)
basement.describe(include="all")

In [ ]:
#Checking features where basement values are not null
basement_not_null = df[~df["BsmtQual"].isnull()]
des = basement_not_null[['Street', 'Utilities','LandSlope', 'Condition2', 'RoofMatl','RoofStyle', 'MasVnrType', 'ExterQual', 'Heating', 'Functional'] + basementVariables].describe(include="object")
display(f'shape of the frame {basement_not_null.shape}')
display(basement_not_null['BsmtQual'].value_counts())
display(basement_not_null['BsmtQual'].value_counts())
display(des)

In [ ]:
df["GarageYrBlt"].median()

In [ ]:
#checking LotFrontage
df["LotFrontage"].describe()

In [ ]:
# Checking Electrical values
df["Electrical"].value_counts()

In [ ]:
# checking fireplace
df['FireplaceQu'].value_counts(dropna=False)

In [ ]:
df['Fireplaces'].value_counts(dropna=False)

In [ ]:
df['MasVnrType'].value_counts(dropna=False)

In [ ]:
df['MasVnrArea'].median()

In [ ]:
garageYearMedian = df["GarageYrBlt"].median()
NoGarageColumns = ["GarageType","GarageFinish","GarageQual","GarageCond"]
df["GarageType"].fillna('Attchd', inplace=True)
df["GarageFinish"].fillna('Unf', inplace=True)
df["GarageQual"].fillna('TA', inplace=True)
df["GarageCond"].fillna('TA', inplace=True)
df["GarageYrBlt"].fillna(garageYearMedian, inplace=True)

NoBasementColumns = ['BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtFinType1', 'BsmtCond']
df['BsmtExposure'].fillna('No', inplace=True)
df['BsmtFinType2'].fillna('Unf', inplace=True)
df['BsmtQual'].fillna('TA', inplace=True)
df['BsmtFinType1'].fillna('Unf', inplace=True)
df['BsmtCond'].fillna('TA', inplace=True)

df["Electrical"].fillna('SBrkr', inplace=True)
df['MasVnrType'].fillna('None', inplace=True)
df['MasVnrArea'].fillna(0)
    
df.loc[df['Fireplaces'] == 0, 'FireplaceQu'] = df.loc[df['Fireplaces'] == 0, 'FireplaceQu'].fillna('NA')
df.loc[df['Fireplaces'] >= 1, 'FireplaceQu'] = df.loc[df['Fireplaces'] >= 1, 'FireplaceQu'].fillna('Gd')

lf_median = df['LotFrontage'].median()
df["LotFrontage"].fillna(lf_median, inplace=True)

In [ ]:
#checking again for null values
display(round(df.isnull().sum()/len(df.index) * 100,2))

In [ ]:
#dropping the small amount of null values remaning
df = df.dropna()

In [ ]:
# converting types
df.MSSubClass = df.MSSubClass.astype(object)

# Label encoding 
df.CentralAir = pd.to_numeric(df.CentralAir.map({"Y":1,"N":0,0:0,1:1}))
df.GarageYrBlt = df.GarageYrBlt.astype(str)
df.YearBuilt = df.YearBuilt.astype(str)
df.YearRemodAdd = df.YearRemodAdd.astype(str)
df.GarageYrBlt = df.GarageYrBlt.astype(str)
df.SalePrice = df.SalePrice.astype("float64")

# transforming the year sales to remove skwewness
df["SalePrice"] = np.log(df["SalePrice"])
minmax = MinMaxScaler()
df["SalePrice"] = minmax.fit_transform(df["SalePrice"].values.reshape(-1, 1))

In [ ]:
df.head()

In [ ]:
df

In [ ]:
# Finding how many numerical and categorical features

num_var = list(df.loc[:,df.dtypes != np.object].columns)
cat_var = list(df.loc[:,df.dtypes == np.object].columns)
print(len(cat_var)," ",len(num_var))

In [ ]:
# Checking correlation
plt.figure(figsize=(50, 50))
sns.heatmap(df.corr(), annot = True, cmap="YlGnBu")

In [ ]:
#univariate_analysis 
#Categorical variables
fig, axis = plt.subplots(21,2,figsize=(20,100))
fig.tight_layout(pad=5.0)
col_number = 0
for i in range(0,21):
    for j in range(0,2):
        if (cat_var[col_number] != "Id"):
            sns.countplot(ax=axis[i,j], x=df[cat_var[col_number]])
            axis[i,j].set_xlabel(cat_var[col_number], fontsize=14);
            axis[i,j].set_ylabel('Count', fontsize=14);
            axis[i,j].set_title(f'Distribution of { cat_var[col_number] }', fontsize=16)
        col_number += 1

plt.show()

In [ ]:
#plotting numeric data to find any pattern
numericalDf = df[num_var]
display(len(numericalDf.columns))
plt.figure(figsize=(200, 200))
sns.pairplot(numericalDf,y_vars=["SalePrice"],x_vars=num_var[0:5])
sns.pairplot(numericalDf,y_vars=["SalePrice"],x_vars=num_var[5:10])
sns.pairplot(numericalDf,y_vars=["SalePrice"],x_vars=num_var[10:15])
sns.pairplot(numericalDf,y_vars=["SalePrice"],x_vars=num_var[15:20])
sns.pairplot(numericalDf,y_vars=["SalePrice"],x_vars=num_var[20:25])
sns.pairplot(numericalDf,y_vars=["SalePrice"],x_vars=num_var[25:30])
sns.pairplot(numericalDf,y_vars=["SalePrice"],x_vars=num_var[30:34])
plt.show()

In [ ]:
# Dropping Features that are highly correlated
cor = df.corr().abs()
upper_triangle = cor.where(np.triu(np.ones(cor.shape), k=1).astype(np.bool))
highCorrelatedFeatures = [col for col in upper_triangle.columns if any(upper_triangle[col] > 0.90)]
if("SalePrice" in highCorrelatedFeatures):
    highCorrelatedFeatures.remove("SalePrice")
df = df.drop(highCorrelatedFeatures,axis=1)
for i in highCorrelatedFeatures:
    num_var.remove(i)

In [ ]:
df.reset_index(inplace=True)
df.head()

In [ ]:
# # creating dummy encoding
# df_dummy = pd.get_dummies(df[cat_var], drop_first=True)
# masterDf = pd.concat([df, df_dummy], axis=1)
# masterDf = masterDf.drop(cat_var,axis=1)
ohc = OneHotEncoder(handle_unknown='ignore', drop='first')
ohc.fit(df[cat_var])
ohc.get_feature_names(cat_var)
encoded_cat_var = pd.DataFrame(ohc.transform(df[cat_var]).toarray(), columns= ohc.get_feature_names(cat_var))
masterDf = pd.concat([df, encoded_cat_var], axis=1)
masterDf = masterDf.drop(cat_var,axis=1)
# separating X and y
y = masterDf.pop("SalePrice")
X = masterDf
encoded_cat_var

In [ ]:
masterDf

In [ ]:
# train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=100)

In [ ]:
X_train

In [ ]:
# Scaling variables
scaler = StandardScaler();
if 'SalePrice' in num_var:
    num_var.remove("SalePrice")
X_train[num_var]= scaler.fit_transform(X_train[num_var]) # fit transform for train data
X_test[num_var] = scaler.transform(X_test[num_var]) # transfrom for test data

In [ ]:
# Hyperparameter tuning using Lasso Regression
params = {'alpha': [0.00019,0.00021,0.00023,0.00025,0.00027,0.00030,0.00033,0.00034,0.00035]}

lasso = Lasso(selection="random")
folds = 5
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1,n_jobs=-1)            
model_cv.fit(X_train, y_train)

In [ ]:
print(model_cv.best_params_)
print(model_cv.best_score_)

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=400]

# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('float32')

# plotting
plt.figure(figsize=(10,10))
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')

plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

In [ ]:
# at alpha = 0.0019, the curve starts to be more flat 
alpha_l =  0.00019
lasso1 = Lasso(alpha = alpha_l)
lasso1.fit(X_train,y_train)
display(lasso1.score(X_train,y_train))
l_train = lasso1.score(X_train,y_train)

In [ ]:
# Residual Analysis
y_train_pred = lasso1.predict(X_train)
res = y_train - y_train_pred
sns.distplot(res)

In [ ]:
# testing on test set
y_test_pred = lasso1.predict(X_test)
l_test = r2_score(y_true=y_test,y_pred=y_test_pred)
l_test

In [ ]:
# Hyperparameter tuning using Ridge Regression
params = {'alpha': [8 ,9,10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 30, 31, 33]}

ridge = Ridge()
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1,n_jobs=-1)            
model_cv.fit(X_train, y_train) 

In [ ]:
print(model_cv.best_params_)
print(model_cv.best_score_)

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=200]

# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('float32')

# plotting
plt.figure(figsize=(10,10))
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')

plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper left')
plt.show()

In [ ]:
# Around 14-15 the test score plateaus
alpha_r =  10
ridge1 = Ridge(alpha = alpha_l)
ridge1.fit(X_train,y_train)
display(ridge1.score(X_train,y_train))
r_train = ridge1.score(X_train,y_train)

In [ ]:
# Residual Analysis
y_train_pred = ridge1.predict(X_train)
res = y_train - y_train_pred
sns.distplot(res)

In [ ]:
# testing on test set
y_test_pred = ridge1.predict(X_test)
r_test = r2_score(y_true=y_test,y_pred=y_test_pred)
r_test

In [ ]:
# we will be using lasso to find predictions
# Loading the test set 
df_t = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

display(df_t.head())

In [ ]:
df_t

In [ ]:
#checking again for null values
display(round(df_t.isnull().sum()/len(df_t.index) * 100,2))

In [ ]:
# Doing all the preprocessing on the test set
Id = df_t["Id"].copy()
df_t = df_t.drop(["PoolQC","Fence","MiscFeature","Alley","Id"],axis=1)
NoGarageColumns = ["GarageType","GarageFinish","GarageQual","GarageCond"]
df_t["GarageType"].fillna('Attchd', inplace=True)
df_t["GarageFinish"].fillna('Unf', inplace=True)
df_t["GarageQual"].fillna('TA', inplace=True)
df_t["GarageCond"].fillna('TA', inplace=True)
df_t["GarageYrBlt"].fillna(0,inplace=True)

NoBasementColumns = ['BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtFinType1', 'BsmtCond']
df_t['BsmtExposure'].fillna('No', inplace=True)
df_t['BsmtFinType2'].fillna('Unf', inplace=True)
df_t['BsmtQual'].fillna('TA', inplace=True)
df_t['BsmtFinType1'].fillna('Unf', inplace=True)
df_t['BsmtCond'].fillna('TA', inplace=True)
    
df_t['FireplaceQu'] = df_t['FireplaceQu'].fillna('No Fireplace')
df_t['Utilities'] = df_t['Utilities'].fillna('AllPub')
df_t['MSZoning'] = df_t['MSZoning'].fillna('RL')
df_t['Exterior1st'] = df_t['Exterior1st'].fillna('VinylSd')
df_t['Exterior2nd'] = df_t['Exterior2nd'].fillna('VinylSd')
df_t['MasVnrType'] = df_t['MasVnrType'].fillna('None')


df_t["LotFrontage"] = df_t["LotFrontage"].fillna(median)
df_t["MasVnrArea"] = df_t["MasVnrArea"].fillna(0.0)

df_t["Electrical"].fillna('SBrkr', inplace=True)
    
df_t.loc[df['Fireplaces'] == 0, 'FireplaceQu'] = df_t.loc[df['Fireplaces'] == 0, 'FireplaceQu'].fillna('NA')
df_t.loc[df['Fireplaces'] >= 1, 'FireplaceQu'] = df_t.loc[df['Fireplaces'] >= 1, 'FireplaceQu'].fillna('Gd')



# converting types
df_t.MSSubClass = df_t.MSSubClass.astype(object)

# Label encoding 
df_t.CentralAir = pd.to_numeric(df_t.CentralAir.map({"Y":1,"N":0,0:0,1:1}))

df_t.GarageYrBlt = df_t.GarageYrBlt.astype(str)
df_t.YearBuilt = df_t.YearBuilt.astype(str)
df_t.YearRemodAdd = df_t.YearRemodAdd.astype(str)
df_t.GarageYrBlt = df_t.GarageYrBlt.astype(str)

df_t = df_t.drop(highCorrelatedFeatures,axis=1)

In [ ]:
df_t

In [ ]:
#checking again for null values
display(round(df_t.isnull().sum()/len(df_t.index) * 100,2))

In [ ]:
df_t["SaleType"].value_counts()

In [ ]:
# Preprocessing the remaning values
zero_value_imputes = ["BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","BsmtFullBath","BsmtHalfBath"]

df_t[zero_value_imputes] = df_t[zero_value_imputes].fillna(0.0)

df_t["KitchenQual"] = df_t["KitchenQual"].fillna("TA")
df_t["Functional"] = df_t["Functional"].fillna("Typ")
df_t["GarageCars"] = df_t["GarageCars"].fillna(2.0)
df_t["SaleType"] = df_t["SaleType"].fillna("WD")
df_t.reset_index(inplace = True)
df_t

In [ ]:
#Creating Dummy variables
df_dummy = pd.DataFrame(ohc.transform(df_t[cat_var]).toarray(), columns= ohc.get_feature_names(cat_var))
masterDf_t = pd.concat([df_t, df_dummy], axis=1)
masterDf_t = masterDf_t.drop(cat_var,axis=1)
masterDf_t[num_var] = scaler.transform(masterDf_t[num_var])

In [ ]:
masterDf_t = masterDf_t.fillna(0)
masterDf_t

In [ ]:
y_test_pred = lasso1.predict(masterDf_t)
y_test_pred = np.reshape(y_test_pred,(-1,1))
display(y_test_pred)

In [ ]:
y_test_pred_value_log = minmax.inverse_transform(y_test_pred)
y_test_pred_values = np.exp(y_test_pred_value_log)

In [ ]:
y_test_pred_values.flatten()
predictions_test = pd.DataFrame({"Id":Id, "SalePrice":y_test_pred_values[:,0]})
predictions_test.to_csv("submission.csv",index=False)
print("Your submission was successfully saved!")